#### Everyone it would seem has been working on a quarantine project to keep them busy and sane these last months. Not to be left out, I am sharing a portion of what I have been working on.

#### I have been interested in making a “pour over” coffee maker for several years now but didn’t want to just throw together some 3D printer looking contraption that would eat up all my precious one-bedroom apartment counter space. Instead I opted to go for the impractical, dream shot coffee maker that would (and this was non-negotiable since all my ex-barista friends would tell me so) make a cup of coffee at exactly the right temperature (measured at the filter), exactly the right bean-to-water ratio (measured to the nearest gram), exactly the right pour pattern (no sprinklers or concentric circles… had to be a spiral), and at exactly the right flow rate (measured to the nearest ml/s). Additionally, the water flow I determined must be laminar when it exited the pour spout just so that it all looked very nice.

#### This was to my naive mind totally do-able. And to an extent it still is, but 1000 dollars and 100+ hrs. later (and still counting) I can say with certainty that it is now unlikely I will ever do better (financially and sanity-wise) than if I had just gone to my friend’s shop (difficult in quarantine) and bought that 5 dollar cup of coffee every day for the rest of my life. But here I am. Soon to have an excellent, hands-free coffee maker but still using my french press every morning...

#### This portion is what I came up with to eliminate a flow meter (not enough room to package in the 2”x4” space available in the tower) from the hot water supply line. It was initially written in Python for easy design visualization. It is, however, totally executable on a C++ Teensy 4.0 (no exclusive NumPy or other non-portable methods used) as it will be implemented in the final coffee maker.


### Import Needed Libraries

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

### Geometry Definitions of Hot Water Tank

In [ ]:
# Tank dimensions
diameter = 2.25

height = 7.5

tower_overHeight = 2.5

width = 2
length = 4
thickness = 0.18

shape = "rectangular"

total_height = tower_overHeight + 8

# Heating element dimensions
cartridge_diameter = 0.5
cartridge_height = 7
wattage = 1500

d = 0.25*2.54/100 # convert to m
length_pipe = 0.3 # 10cm
Cv = 4.8
rho = 965.31 # density of water kg/m^3 at 90degC
mu = 0.0003142 # viscosity of water at 90degC
xi = 4.08
grav = 9.81
relative_roughness = 0.005

### Thermodynamic Requirements

#### Next the machine will determine the fractions of water to be delivered to the filter at each of the 4 stages of coffee filtering. Please see Blue Bottle tips (https://bluebottlecoffee.com/preparation-guides/pour-over).


In [ ]:
pour = [60, 90, 100, 100]
rest = [30, 45, 45, 45]
time = [15, 15, 20, 20]

setting = "double"

# Only if trying for double cup performance
if setting == "double":
    pour = [i * 2 for i in pour]
    rest = [i * 1.5 for i in rest]
    time = [i * 2 for i in rest]


m = np.zeros(len(pour))
m[0] = pour[0]

for i in range(len(m)-1):
    m[i+1] = pour[i+1] - ((wattage * (rest[i]-pour[i]*60/400)) / (4.19 * 75)) # leftover mass
    #print((wattage * rest[i]) / (4.19 * 75))

min_volume = sum(m)
print(pour)

print("Minimum starting water volume:")
print(min_volume)

### Compute Usable Internal Volume in Water Tank

In [ ]:
if shape == "rectangular":
    volume = (width - 2*thickness) * 2.54 * (length - 2*thickness) * 2.54 * height * 2.54
else:
    volume = math.pi*((diameter*2.54/2)**2)*(height*2.54)

        
cartridge_volume = math.pi*((cartridge_diameter*2.54/2)**2)*(cartridge_height*2.54)

empty_volume = volume - cartridge_volume

upper_volume = volume * (1 - cartridge_height/height)
lower_volume = empty_volume - upper_volume

flowable_volume = lower_volume * (1 - (height - tower_overHeight)/cartridge_height) + upper_volume
unflowable_water = lower_volume * (height - tower_overHeight)/cartridge_height
req_water = min_volume + lower_volume * (height - tower_overHeight)/cartridge_height

print("Total Volume:")
print(empty_volume)

print("Flowable Volume:")
print(flowable_volume)

print("Minimum Required Water:")
print(req_water)

print("Unusable Water:")
print(unflowable_water)


### Initialize Variables for Active Pour Time-Step Iteration Sequence

In [ ]:
flow_rate = np.zeros(1000)
flow_rate[0] = 1

current_water = np.zeros(1000)
current_water[0] = start_water_volume #defined in millimeters

water_height = np.zeros(1000)

actual_pour = np.zeros(len(m))

Q = np.zeros(1000)
v = np.zeros(1000)
Pp = np.zeros(1000)
Pf = np.zeros(1000)
diff = np.zeros(1000)
friction_losses = np.zeros(1000)

i = 0

### Report Active Design Constraints To User

In [ ]:
if req_water < lower_volume:
    start_water_volume = lower_volume
    print("Adjusted water to submerge heating element.")
    print("New starting water volume:")
    print(start_water_volume)
else:
    start_water_volume = req_water
    print("Water volume minimum amount driven by wattage of heater.")
    print("Starting water volume:")
    print(start_water_volume)

additional_volume = empty_volume - start_water_volume
print("Additional Volume Available: ",additional_volume)

added_water = 0
# add water to initial water to increase head height
if added_water > 0:
    start_water_volume = start_water_volume + added_water
    print("New starting water volume (user added):", start_water_volume)
    
    if current_water[0] >= lower_volume:
    water_height[0] = cartridge_height + (current_water[0]-lower_volume)/upper_volume * (height - cartridge_height)
else:
    print("Heater not submerged!")
    water_height[0] = (current_water[0]/lower_volume) * cartridge_height
    
print("Starting water height:",water_height[0])

if cartridge_height >= height:
    print("Check cartridge height interference.")
    
if start_water_volume > empty_volume:
    print("Need a bigger tank.")
    
if total_height > 17:
    print("Coffee machine is too tall.")

### Iterate

In [ ]:
for j in range(len(m)):
    
    if j > 0:
        # add water for filling up tank
        current_water[i] = current_water[i] + (pour[j] - m[j])
        # check if the added water would increase volume to something larger than the tank can accomodate
        if current_water[i] > empty_volume:
            # if upper limit of newly heated water is amount is too large decrease to fit in container (could also decrease wattage)
            current_water[i] = empty_volume - 15
            #print("Heater more than powerful - reduced incoming water.")
    
    interval_water = current_water[i]
    
    if current_water[i] >= lower_volume:
        water_height[i] = cartridge_height + (current_water[i]-lower_volume)/upper_volume * (height - cartridge_height)
    else:
        # water does not fully submerge heating element!
        print("Heater not fully submerged!")
        water_height[i] = (current_water[i]/lower_volume) * cartridge_height
        
    head_water_height = tower_overHeight - height + water_height[i]
    
    k = 0 # stand in for actual time - each step is assumed to be 1 sec

    while k < 2*time[j] and interval_water - current_water[i] < pour[j] and head_water_height > 0.01:

        count = 0 # counter to limit the number of times the code will iterate on a single time step
        
        Q = 1 # initial flow rate to pass thru the while logic check
        
        if k > 0:
            Q_prev = flow_rate[i-1] # use previous time step flow rate to be closer to likely converged solution at current timestep
        else:
            Q_prev = 5 # if this is the first time step, use a different Q_prev value to get thru the while logic
            
        head_water_height = tower_overHeight - height + water_height[i] # head water height at a time step will be constant
        head_pressure = rho * grav * head_water_height * 2.54/100/6894.76 # only need to calculate once
                
        unit_add = -0.5 # convergence varaible used to "narrow in" on the converged flow rate. Later halved and inverted. unit [ml/s]
                
        while abs(Q - Q_prev) > 0.01 and count < 200:

            count += 1 # count used to prevent unsolvable, divergent, or very unstable flow points from preventing solution report

            diff[count] = abs(Q - Q_prev) # flow rate comparison between previous flow rate solution and current time-step temporary solution. Physical problem states this would not likely be large step jump.
            
            if diff[count] > diff[count-1]: # if current solution results in larger difference, "zero" of function passed and need to reverse convergence direction and iterate over smaller changes
                # flip directions and tighten increment
                unit_add = unit_add * -0.5
                
            Q_prev = Q_prev + unit_add

            v = Q_prev/1000000 * 4 / (math.pi * (d**2)) # calculate flow velocity at exit of pipe

            Re = d * v * rho / mu # calculate internal pipe Reynolds Number

            a = 1 / (1 + (Re/2712)**8.4) # coefficient 
            b = 1 / (1 + (Re/(150/relative_roughness))**1.8) # coefficient

            f = (64/Re)**a * (0.75*np.log(Re/5.37))**(2*(a-1)*b) * (0.88 * np.log(6.82/relative_roughness))**(2*(a-1)*(1-b)) # friction factor

            Pp = f * length_pipe/d * rho/2 * v**2

            Pf = xi *  rho/2 * v**2

            friction_losses = (Pf + Pp) * 0.000145038

            if friction_losses < head_pressure:
                Q = Cv * math.sqrt(head_pressure - friction_losses) * (3785.41/60) # compute flow rate throught the solenoid based on head pressure losses in pipe and through restriction of solenoid
                
        flow_rate[i] = Q # store flow rate

        current_water[i+1] = current_water[i] - flow_rate[i] # recalculate water volume based on flow rate out of pipe

        if current_water[i] >= lower_volume: # calculate new water height in tank
            water_height[i+1] = cartridge_height + (current_water[i]-lower_volume)/upper_volume * (height - cartridge_height)
        else:
            water_height[i+1] = (current_water[i]/lower_volume) * cartridge_height

        i += 1
        k += 1

    actual_pour[j] = interval_water - current_water[i]

    print("Pour volume [ml]",j+1,":",actual_pour[j])
    print("Length of Pour [s]:",k)
    if k > time[j]:
        print("Increase head height for higher water flow.")
    if actual_pour[j] < pour[j]:
        print("Not enough water poured in twice the time!!")

### Plot Process Data

In [ ]:
plt.plot(water_height - (height - tower_overHeight))
plt.plot([0, 100],[cartridge_height- (height - tower_overHeight),cartridge_height- (height - tower_overHeight)],'--')
plt.xlim(0, 100)
plt.show()

plt.plot(current_water[0]-current_water)
plt.plot([0,15],[60,60],'--')
plt.plot([15,15],[0,60],'--')
plt.xlim(0, 100)
plt.show()